# Show used config values

In [ ]:
from collections import defaultdict
import os
import json
from helpers import RESULTS_DIR

# Show possible values for the config files
config_values = defaultdict(set)
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    for key, value in config.items():
        config_values[key].add(str(value))

# Drop experiment name, best, mean, std
ignore = ['experiment_name']
for fitness_method in ['default', 'balanced', 'defensive']:
    ignore.extend([f'best_{fitness_method}', f'mean_{fitness_method}', f'std_{fitness_method}', f'Q5_{fitness_method}', f'Q95_{fitness_method}'])
for key in ignore:
    config_values.pop(key, None)
display(config_values)

# Count runs per config

In [ ]:
from collections import defaultdict
import os
import json
import shutil
from helpers import RESULTS_DIR

ignore = ['experiment_name']
for fitness_method in ['default', 'balanced', 'defensive']:
    ignore.extend([f'best_{fitness_method}', f'mean_{fitness_method}', f'std_{fitness_method}', f'Q5_{fitness_method}', f'Q95_{fitness_method}'])

# Count number of runs per unique config
runs_per_config = defaultdict(int)
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    for key in ignore:
        config.pop(key, None)
    if config['gen'] < config['gens']-1:
        print(f'Incomplete run: {folder}. {config["gen"]} out of {config["gens"]} generations.')
        # Delete folder and contents
        shutil.rmtree(os.path.join(RESULTS_DIR, folder))
        continue
    runs_per_config[str(config)] += 1

# Show number of runs per unique config
for config, runs in sorted(runs_per_config.items(), key=lambda x: x[0], reverse=True):
    print(f'{runs} runs for config {config}')

# File to show plots for the evolution runs

In [ ]:
from plotting import create_plot
from helpers import find_folders, RESULTS_DIR

default_config = { # First experiment with 10 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "default",
    "pick_parent_method": "tournament",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}

default_config = { # Second experiment with 30 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}

# variable = {"fitness_method": ["default", "balanced"]}  	        # seems pretty much the same
# variable = {"fitness_method": ["default", "rank"]}  	            # pretty much the same
# variable = {"normalization_method": ["default", "domain_specific"]} # seems consistently slightly worse (puzzling)
# variable = {"normalization_method": ["default", "around_0"]}        # mean is similar, best is sometimes better, sometimes worse, 
# variable = {"pick_parent_method": ["tournament", "greedy"]}         # way better mean obviously, inconsistent best
# variable = {"pick_parent_method": ["tournament", "multinomial"]}    # multinomial seems consistently better
# variable = {"survivor_method": ["multinomial", "greedy"]}           # way better mean obviously, inconsistent best
# variable = {"survivor_method": ["multinomial", "tournament"]}           # tournament seems consistently worse
# variable = {"crossover_method": ["none", "default"]}                # consistently worse mean, similar best
# variable = {"crossover_method": ["none", "ensemble"]}               # consistently worse mean, inconsistent best
# variable = {"randomini": ["no", "yes"]}                           # just luck based
# variable = {"multi_ini": [False, True]}                           # obviously worse scores but better for multi ini eval
variable = {"mutation_type": ["normal", "stochastic_decaying"]}   # better mean, maybe slightly better best

for enemies in [[1], [2], [3], [4], [5], [6], [7], [8]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})

        folders_list.append(find_folders(config))

    create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default')


# Boxplots

number of wins/defeats isn't great as a metric for a boxplot because it only ranges between 0 and 8

In [ ]:
# Boxplots
from plotting import create_boxplot
from helpers import find_folders, RESULTS_DIR

default_config = {
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "default",
    "pick_parent_method": "tournament",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}
default_config = {
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}
# variable = {"fitness_method": ["default", "balanced"]} # not much difference
# variable = {"normalization_method": ["default", "domain_specific"]} # not much difference
# variable = {"pick_parent_method": ["tournament", "multinomial"]}
# variable = {"multi_ini": [False, True]} # obviously much better scores for multi ini eval
variable = {"mutation_type": ["normal", "stochastic_decaying"]} # sometimes slightly better, sometimes slightly worse
# variable = {"crossover_method": ["none", "ensemble"]} # sometimes slightly better, sometimes slightly worse

# Same as above but with boxplots
for enemies in [[1], [2], [3], [4], [5], [6], [7], [8]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})
    
        folders_list.append(find_folders(config))

    create_boxplot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, randomini_eval=False, multi_ini_eval=False)

# Hardest enemies

In [ ]:
# Make boxplot for each enemy based on the best fitness in each run
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from helpers import find_folders, RESULTS_DIR

def create_enemy_boxplot(config={}, randomini_eval=False, multi_ini_eval=False, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain'):

    add_str = ""
    if randomini_eval:
        add_str = "_randomini"
    elif multi_ini_eval:
        add_str = "_multi-ini"

    data = []
    shortest_folders_len = 1e6
    for enemy in range(1,9):
        config.update({"enemies": [enemy]})
        folders = find_folders(config)
        shortest_folders_len = min(shortest_folders_len, len(folders))

        if not folders:
            print(f'No folders found for enemy: {enemy}')
            return


        runs = []
        for folder in folders:
            # Create empty df with columns for [gain, fitness, fitness_balanced, n_wins]
            df = pd.DataFrame(columns=['gain', 'fitness', 'fitness_balanced', 'wins'])

            # Read results from eval_best.json
            with open(f'{results_dir}/{folder}/eval_best{add_str}.json', 'r') as f:
                saved = json.load(f)
            df = pd.DataFrame(saved["results"])
            # Turn wins list into number of wins if wins is a list type
            if type(df['wins'][0]) == list:
                df['wins'] = df['wins'].apply(lambda x: sum(x))

            # Average over the 5 evals, keep dims
            df = df.mean(axis=0)
            df = df.to_frame().transpose()
            runs.append(df)
        runs = pd.concat(runs, axis=0)

        data.append(runs[metric])
    print(f'Shortest folder list: {shortest_folders_len}')
    plt.figure(figsize=figsize)

    # Plot boxplot(s)
    plt.boxplot(data)
    plt.xticks(range(1,9), range(1,9))
    plt.title(f'{metric.capitalize()} boxplot')

    plt.xlabel('Enemy')
    plt.ylabel(metric.capitalize())
    if save_png:
        if not os.path.exists(f'plots/{str(variable)}'):
            os.makedirs(f'plots/{str(variable)}')
        plt.savefig(f'plots/{str(variable)}/{metric}_boxplot.png')
    plt.show()

print('Evaluated on all initial enemy positions')
create_enemy_boxplot(config={
        "randomini": "no",
        "multi_ini": False,
    },
    randomini_eval=False, multi_ini_eval=True, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain')

print('Evaluated on only default enemy position')
create_enemy_boxplot(config={
        "randomini": "no",
        "multi_ini": False,
    },
    randomini_eval=False, multi_ini_eval=False, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain')

# Delete folders without config

In [ ]:
import os
import json
from helpers import RESULTS_DIR

# Get all folders
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    # Check if config.json exists
    if not os.path.exists(os.path.join(RESULTS_DIR, folder, 'config.json')):
        print(f'No config.json in folder: {folder}, deleting folder')
        for file in os.listdir(os.path.join(RESULTS_DIR, folder)):
            os.remove(os.path.join(RESULTS_DIR, folder, file))
        os.rmdir(os.path.join(RESULTS_DIR, folder))
        continue
    # else:
    #     with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
    #         config = json.load(f)
    #     if "best_log" not in config:
    #         print(f'No best_log in config.json in folder: {folder}, deleting folder')
    #         # Delete folder and contents
    #         for file in os.listdir(os.path.join(RESULTS_DIR, folder)):
    #             os.remove(os.path.join(RESULTS_DIR, folder, file))
    #         os.rmdir(os.path.join(RESULTS_DIR, folder))
    #         continue

# Update all configs

In [ ]:
import os
import json
from helpers import RESULTS_DIR

# Add variable to all config.json files
variable = "exploration_island"
default_value = False

folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    if variable not in config:
        config.update({variable: default_value})
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)

# Multi Enemy

In [ ]:
# Make matplotlib plots interactive
%matplotlib widget

In [ ]:
from plotting import create_plot
from helpers import find_folders, RESULTS_DIR

default_config = { # Second experiment with 30 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "stochastic_decaying",
    "domain_upper": 1,
    "domain_lower": -1,
}


variable = {"mutation_type": ["normal", "stochastic_decaying"]}   # better mean, maybe slightly better best
variable = {"exploration_island": [
    False, 
    True
]}

for enemies in [[1,2,3,4,5,6,7,8],[3,4,6,7]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})

        folders_list.append(find_folders(config))

    create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default', plot_separate_lines=True)


# Exploration Island

In [ ]:
from plotting import create_plot
from helpers import find_folders, RESULTS_DIR

default_config = { # Second experiment with 30 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "stochastic_decaying",
    "domain_upper": 1,
    "domain_lower": -1,
}

variable = {"exploration_island": [
    False, 
    True
]}

# for enemies in [[1], [2], [3], [4], [5], [6], [7], [8]]:
for enemies in [[1, 2, 3, 4, 5, 6, 7, 8]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})

        folders_list.append(find_folders(config))

    create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default', exploration_island=True, plot_separate_lines=True)
    # create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default_explore', exploration_island=True)

# Landscape Data

In [ ]:
import os
import numpy as np

results_dir = 'landscape_data'

# Get all folders
folders = [f for f in os.listdir(results_dir) if os.path.isdir(os.path.join(results_dir, f))]

# Folders contain pop.npy and results.npy
# pop.npy contains the population as a (pop_size, n_params) array
# results.npy contains the results as a (pop_size, n_enemies, 3) array where the last dimension is [player life, enemy life, time]
# First concatenate all pop.npy and results.npy files into one big array where pop_size is now the combined pop_size of all runs
rand_pop = []
rand_results = []
for folder in folders:
    pop = np.load(os.path.join(results_dir, folder, 'pop.npy'))
    results = np.load(os.path.join(results_dir, folder, 'results.npy'))
    rand_pop.append(pop)
    rand_results.append(results)
rand_pop = np.concatenate(rand_pop, axis=0)
rand_results = np.concatenate(rand_results, axis=0)

# Get indices where time != -1 (not yet evaluated)
indices = np.where(rand_results[:,0,2] != -1)
# Filter out those indices
rand_pop = rand_pop[indices]
rand_results = rand_results[indices]

# Now we have the total population and results, we can calculate the gain for each individual
# The gain is the sum of the player life for each enemy minus the sum of the enemy life for each enemy
# The gain per enemy is a (pop_size, n_enemies) array
rand_gain_per_enemy = rand_results[:,:,0] - rand_results[:,:,1]
rand_gain = np.sum(rand_gain_per_enemy, axis=1)

# Sort the gain array and get the indices
indices = np.argsort(rand_gain)[::-1]
rand_pop = rand_pop[indices]
rand_results = rand_results[indices]
rand_gain_per_enemy = rand_gain_per_enemy[indices]
rand_gain = rand_gain[indices]

print('Population size:', rand_pop.shape)
print('Gain sorted:')
display(rand_gain.tolist())
print('Gain per enemy sorted by total gain:')
display(rand_gain_per_enemy)

In [ ]:
from beepy import beep

beep(sound=1)

In [ ]:
# Create a population of the best individuals for each run
from helpers import RESULTS_DIR
from tqdm import tqdm
import os
import json
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]

# These folders each contain one best individual, saved in best.txt
# The results for this best individual are saved in eval_best_all-enemies.json
# This json file is structured as follows:
# {
#     "results": [
#         {
#             "gains": [gain1, gain2, ..., gain8],
#         }
#     ]
# }
# We'll use this to construct the gain_per_enemy array
best_pop = []
best_results = []
enemy_sets = []
folders_without_eval = 0
for folder in tqdm(folders):
    if not os.path.exists(os.path.join(RESULTS_DIR, folder, 'eval_best_all-enemies.json')):
        folders_without_eval += 1
        continue
    with open(os.path.join(RESULTS_DIR, folder, 'eval_best_all-enemies.json'), 'r') as f:
        saved = json.load(f)
    best_results.append(saved['results'][0]['gains'])
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    enemy_sets.append(str(config['enemies']))
    best_pop.append(np.loadtxt(os.path.join(RESULTS_DIR, folder, 'best.txt')))
best_pop = np.array(best_pop)
best_gain_per_enemy = np.array(best_results)
best_gain = np.sum(best_gain_per_enemy, axis=1)

# Make array out of enemy sets by keeping a dictionary of enemy sets and a corresponding integer
enemy_set_dict = {
    '[1, 2, 3, 4, 5, 6, 7, 8]': 0,
    '[1]': 1,
    '[2]': 2,
    '[3]': 3,
    '[4]': 4,
    '[5]': 5,
    '[6]': 6,
    '[7]': 7,
    '[8]': 8,
}
starting_int = 9
for e_set in set(enemy_sets):
    if e_set not in enemy_set_dict:
        enemy_set_dict[e_set] = starting_int
        starting_int += 1
rev_enemy_set_dict = {v: k for k, v in enemy_set_dict.items()}
print(enemy_set_dict)

enemy_sets_array = np.array([enemy_set_dict[e_set] for e_set in enemy_sets])

# Sort the gain array and get the indices
indices = np.argsort(best_gain)[::-1]
best_pop = best_pop[indices]
best_gain_per_enemy = best_gain_per_enemy[indices]
best_gain = best_gain[indices]
enemy_sets_array = enemy_sets_array[indices]

print(f'Folders without eval: {folders_without_eval}')

print('Best Population size:', best_pop.shape)
print('Gain sorted:')
enemies_beaten = np.sum(best_gain_per_enemy > 0, axis=1)
which_enemies_beaten = np.where(best_gain_per_enemy > 0, 1, 0)
display([f'Gain: {best_gain[i]:.0f}, {enemies_beaten[i]} beaten: {which_enemies_beaten[i]}. Evo on enemies: {rev_enemy_set_dict[enemy_sets_array[i]]}' for i in range(len(best_gain))])
print('Gain per enemy sorted by total gain:')
display(best_gain_per_enemy)

In [ ]:
# Dimensionality reduction
# from sklearn.manifold import MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# List of 10 colors
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'w', 'orange', 'purple']

# Reduce dimensionality of total_pop to 2 dimensions
# mds = MDS(n_components=2)
# total_pop_reduced = mds.fit_transform(total_pop)
pca = PCA(n_components=2)
rand_and_best_pop = np.concatenate([rand_pop, best_pop], axis=0)
rand_and_best_pop_reduced = pca.fit_transform(rand_and_best_pop)
rand_pop_reduced = rand_and_best_pop_reduced[:rand_pop.shape[0]]
best_pop_reduced = rand_and_best_pop_reduced[rand_pop.shape[0]:]

indices_for_best = np.where(enemy_sets_array == 0)
best_pop_reduced_all_e = best_pop_reduced[indices_for_best]

exponent = 1.5
rand_alpha = ((rand_gain + 800) / 1600)**exponent
best_alpha = ((best_gain[indices_for_best] + 800) / 1600)**exponent

# Plot the reduced population
plt.style.use('dark_background')
plt.figure(figsize=(5,5))
# plt.scatter(rand_pop_reduced[:,0], rand_pop_reduced[:,1], alpha=rand_alpha, c=rand_alpha, cmap='viridis', s=1)
# plt.scatter(rand_pop_reduced[0,0], rand_pop_reduced[0,1], c='w', s=10, marker='o')
for i, color in zip(range(1,8), colors):
    indices = np.where(enemy_sets_array == i)
    plt.scatter(rand_pop_reduced[indices,0], rand_pop_reduced[indices,1], c=color, s=1)
# plt.scatter(best_pop_reduced_all_e[:,0], best_pop_reduced_all_e[:,1], alpha=best_alpha, c=best_alpha, cmap='viridis', s=5)
plt.title('Dim Reduced population')
plt.xlabel('dim 1')
plt.ylabel('dim 2')
plt.show()

In [ ]:
# Dimensionality reduction
# from sklearn.manifold import MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# List of 10 colors
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'w', 'orange', 'purple']

# Reduce dimensionality of total_pop to 2 dimensions
# mds = MDS(n_components=2)
# best_pop_reduced = mds.fit_transform(best_pop)
pca = PCA(n_components=2)
best_pop_reduced = pca.fit_transform(best_pop)

# Plot the reduced population
plt.style.use('dark_background')
plt.figure(figsize=(5,5))
for i, color in zip(range(1,9), colors):
    indices = np.where(enemy_sets_array == i)
    plt.scatter(best_pop_reduced[indices,0], best_pop_reduced[indices,1], c=color, s=1)
plt.title('Dim Reduced population')
plt.xlabel('dim 1')
plt.ylabel('dim 2')
plt.legend(['Enemy 1', 'Enemy 2', 'Enemy 3', 'Enemy 4', 'Enemy 5', 'Enemy 6', 'Enemy 7', 'Enemy 8'])
plt.show()

# There seems to be no correlation between the enemy that was trained on and the position in the reduced space